# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#import current weather data from csv
clean_weather_data = pd.read_csv('../Output/clean_weather_data.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = clean_weather_data[["Latitude", "Longitude"]]
humidity = clean_weather_data['Humidity (%)']

fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

### Narrowing Data to My Ideal Weather Conditions
My ideal weather conditions include:
- Anywhere between 75 and 80 degrees.
- A pleasent breeze (5 - 24 mph)
- Some clouds in the sky to admire (20% to 50%)

In [68]:
# narrowing the data to my ideal weather conditions
ideal_weather_data = clean_weather_data.loc[(clean_weather_data['Temperature (F)'] >= 75) & (clean_weather_data['Temperature (F)'] < 80),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Wind Speed (mph)'] >= 5) & (ideal_weather_data['Wind Speed (mph)'] < 24),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Cloudiness (%)'] >= 20) & (ideal_weather_data['Cloudiness (%)'] < 50),:]

ideal_weather_data = ideal_weather_data.set_index('City')
ideal_weather_data

,Latitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City,,,,,,,
atuona,-9.8000,-139.0333,77.68,77.68,79,23,21.27
albany,42.6001,-73.9662,75.70,80.47,73,22,5.41
faanui,-16.4833,-151.7500,79.05,79.05,77,22,12.37
itarema,-2.9248,-39.9167,79.38,79.38,78,20,18.37
touros,-5.1989,-35.4608,77.90,77.90,83,35,14.70
teahupoo,-17.8333,-149.2667,78.76,78.76,77,31,9.46
denpasar,-8.6500,115.2167,78.46,78.46,94,20,9.22
ambur,12.7833,78.7000,76.84,76.84,79,43,5.97
harper,4.3750,-7.7169,75.67,75.67,87,44,10.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [63]:
hotel_df = pd.DataFrame(columns = ['Hotel Name', 'Lat', 'Lng'])

In [61]:
# test_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=29.4241,-98.4936&radius=5000&type=hotel&key={g_key}'
test_response = requests.get(test_url).json()
test_response['results'][1]['geometry']['location']['lng']



-98.492007

In [65]:
# find the closest restaurant of each type to coordinates

search_radius= 5000
search_type = 'hotel'

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
query_perams = f'&radius={search_radius}&type={search_type}&key={g_key}'

# use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_data.iterrows():
  
    lat = row['Latitude']
    lng = row['Longitude']
        
    # get location type from df
    location = f'location={lat},{lng}'
    
    # assemble url and make API request
    print(f"Searching for Hotel in {index}.")
    query_url = base_url + location + query_perams
    response = requests.get(query_url).json()   
    
    if response['status'] == 'OK':
    
        # results code stem
        results = response['results'][1]
     
        try: 
            hotel_df.loc[index, 'Hotel Name'] = results['name']
            hotel_df.loc[index, 'Lat'] = results['geometry']['location']['lat']
            hotel_df.loc[index, 'Lng'] = results['geometry']['location']['lng']
        
        except (KeyError, IndexError):
            print("No Hotels Found Nearby... Skipping.")
    else:
        print("No Hotels Found Nearby... Skipping.")

    print("------------")

Searching for Hotel in atuona.
------------
Searching for Hotel in albany.
------------
Searching for Hotel in faanui.
------------
Searching for Hotel in itarema.
------------
Searching for Hotel in touros.
------------
Searching for Hotel in teahupoo.
------------
Searching for Hotel in denpasar.
------------
Searching for Hotel in ambur.
------------
Searching for Hotel in harper.
------------
Searching for Hotel in ushtobe.
------------
Searching for Hotel in labuhan.
------------


In [66]:
hotel_df

,Hotel Name,Lat,Lng
atuona,Banque Socredo,-9.803691,-139.042298
albany,WGNA-FM Albany,42.637022,-73.997072
faanui,Bora-Bora Pearl Beach Resort & Spa,-16.475941,-151.772479
itarema,"Sea Lounge! Bar, Club & Restaurant",-2.881102,-39.90811
touros,INN NEW HORIZON,-5.204428,-35.461233
teahupoo,Vanira Lodge,-17.84351,-149.267024
denpasar,ASTON Denpasar Hotel & Convention Center,-8.63932,115.206122
ambur,Ramu Offset Printers,12.786227,78.716594
harper,EVA MARKETING,4.377218,-7.717011
ushtobe,Halyk Bank,45.254555,77.988411


In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=None, display_info_box=None)
fig1.add_layer(marker_layer)

fig1

# Display figure
